In [ ]:
"""
Created on Thu Nov 2 11:19:21 2021
@author: cliu
"""
import os
import yaml
import datetime as dt
import pandas as pd
import numpy as np
from bisect import *
import random
import ast
_join = os.path.join

CONFIG = r'T:/ABM/ABM_FY22/RSM/VisionEval/Model/Update_Automations/settings.yml'
with open(CONFIG) as cff:
    config =yaml.safe_load(cff)
driverHTSBin = config['driverHTSBin']
scenpath = ''.join(config['scenpath'])
target_path = ''.join(config['target_path'])

In [ ]:
# 1 ToursByHh_df.csv
df_tour = pd.read_csv(_join('HTS', 'tours_debug_newid.csv'))
df_hh = pd.read_csv(_join('HTS', 'SDRTS_Household_Data_20170731 TAZ.csv'))
df_trip_hts = pd.read_csv(_join('HTS', 'SDRTS_Trip_Data_20170731 TAZ.csv'))[['hhid', 'pernum', 'tripnum','travelers_total','mode1']] 
df_trip_rsg = pd.read_csv(_join('HTS', 'trips_debug_newid.csv'))[['HH_ID','PER_ID','TOUR_ID','TRIP_ID', 'ORIG_PLACENO', 'TRIPMODE','puid', 'mode_adj']]
df_veh_hts = pd.read_csv(_join('HTS', 'SDRTS_Vehicle_Data_20170731(2016HTS).csv'))[['hhid', 'vehnum', 'category']]
df_mode = pd.read_csv(_join('NHTS2001 analysis', 'mode share.csv'))

df_tour = df_tour.merge(df_hh[['hhid', 'home_address_TAZ']], left_on='HH_ID', right_on='hhid', how="left")
df_tour = df_tour.fillna(0)

# estimating TravelTime, DwellTime, StartHome, EndHome
df_tour['DwellTime'] =  df_tour['OSTOP_1_DUR_HR']*60 + df_tour['OSTOP_1_DUR_MIN'] + \
                        df_tour['OSTOP_2_DUR_HR']*60 + df_tour['OSTOP_2_DUR_MIN'] + \
                        df_tour['OSTOP_3_DUR_HR']*60 + df_tour['OSTOP_3_DUR_MIN'] + \
                        df_tour['OSTOP_4_DUR_HR']*60 + df_tour['OSTOP_4_DUR_MIN'] + \
                        df_tour['ISTOP_1_DUR_HR']*60 + df_tour['ISTOP_1_DUR_MIN'] + \
                        df_tour['ISTOP_2_DUR_HR']*60 + df_tour['ISTOP_2_DUR_MIN'] + \
                        df_tour['ISTOP_3_DUR_HR']*60 + df_tour['ISTOP_3_DUR_MIN'] + \
                        df_tour['ISTOP_4_DUR_HR']*60 + df_tour['ISTOP_4_DUR_MIN'] + \
                        df_tour['PRIMDEST_DEPART_HOUR']*60 + df_tour['PRIMDEST_DEPART_MIN'] - \
                        df_tour['PRIMDEST_ARRIVE_HOUR']*60 - df_tour['PRIMDEST_ARRIVE_MIN']
df_tour['TravelTime'] = df_tour['TOUR_DUR_HR']*60 + df_tour['TOUR_DUR_MIN'] - df_tour['DwellTime']  
df_tour.loc[df_tour['home_address_TAZ'] == df_tour['OTAZ'], 'StartHome'] = 'TRUE'
df_tour['StartHome'] = df_tour['StartHome'].fillna('FALSE')
df_tour.loc[df_tour['home_address_TAZ'] == df_tour['DTAZ'], 'EndHome'] = 'TRUE'
df_tour['EndHome'] = df_tour['EndHome'].fillna('FALSE')

# estimating trips and persons (travelers_total)
df_trip_rsg = df_trip_rsg.merge(df_trip_hts, left_on=['HH_ID','PER_ID','ORIG_PLACENO'], \
                                right_on=['hhid', 'pernum', 'tripnum'], how='left')
df_trip_rsg2 = df_trip_rsg.groupby(['HH_ID','PER_ID','TOUR_ID'])['TRIP_ID', 'travelers_total'].max().reset_index()
df_trip_rsg2.rename(columns={'TRIP_ID':'Trips'}, inplace=True) 
df_tour2 = df_tour.merge(df_trip_rsg2[['HH_ID','PER_ID','TOUR_ID','Trips','travelers_total']],\
                         on=['HH_ID','PER_ID','TOUR_ID'], how="left")
df_tour2 = df_tour2.merge(df_trip_rsg[['HH_ID','PER_ID','TOUR_ID','TRIP_ID','mode1','mode_adj']], 
                          on=['HH_ID','PER_ID','TOUR_ID'], how="left")
df_tour2 = df_tour2[df_tour2['TRIP_ID'] == 1] # only keeps the first trip leg for each tour

# estimating Vehtype, HhVehUsed
df_tour2.loc[(df_tour2['TOURMODE']<=3) & (df_tour2['mode1']>=6) & (df_tour2['mode1']<=16), 'vehnum'] = df_tour2['mode1']-5\
                            # auto mode to Auto, LtTrk, OtherTrk, Motorcycle. TOURMODE-HTS, mode1-HTS trip mode(first mode)
df_tour2 = df_tour2.merge(df_veh_hts, on=['hhid','vehnum'], how='left')
df_tour2['Vehtype'] = df_tour2['category'].replace({'car': 1, 'light truck': 4, 'Motorcycle': 7, 'OthTrk': 5})
df_tour2.loc[(df_tour2['TOURMODE']<=3) , 'HhVehUsed'] = 2
df_tour2.loc[(df_tour2['TOURMODE']<=3) & (df_tour2['vehnum']>0), 'HhVehUsed'] = 1

# converting HTS tour mode to NHTS tour mode (Trptrans)
df_tour2['Trptrans'] = df_tour2 ['TOURMODE']
df_tour2['Trptrans'] = df_tour2['Trptrans'].replace({10:22})
df_tour2['Trptrans'] = df_tour2['Trptrans'].replace({2: 1, 3: 1, 4: 26, 5: 25, 6: 10, 7: 10, 8: 10, 9: 10, 11: 12})
df_tour2.loc[(df_tour2['Vehtype']>0), 'Trptrans'] = df_tour2['Vehtype'] # update auto trip modes if household vehicle is used
# 39-LRT:18-StreetCar, 41-RailIntercity/42-Rail Other:15-Train
df_tour2.loc[(df_tour2['mode_adj']==39) & (df_tour2['Trptrans']==10), 'Trptrans'] = 18
df_tour2.loc[(df_tour2['mode_adj']==41) & (df_tour2['Trptrans']==10), 'Trptrans'] = 15
df_tour2.loc[(df_tour2['mode_adj']==42) & (df_tour2['Trptrans']==10), 'Trptrans'] = 15
df_tour2.loc[(df_tour2['mode_adj']==150) & (df_tour2['Trptrans']==10), 'Trptrans'] = 16

df_tour2['Vehtype'] = df_tour2['Vehtype'].fillna('NA')
df_tour2['HhVehUsed'] = df_tour2['HhVehUsed'].fillna('NA')

# Approximate Vehid, Disttowk, Whyto - these attributes are not in used in VERSPM modules
df_tour2['Vehid'] = 0
df_tour2['Disttowk'] = 0
df_tour2['Whyto'] ='1-1'

In [ ]:
# 2 Imputation of auto tour mode if household vehichle is not used (auto, light track, other truck, RV, motorcycel)
df_tour2['IsDist20'] = '2'
df_tour2.loc[df_tour2['DIST']<=20, 'IsDist20'] = '1'
#df_tour2['IsTrips2'] = '2'
#df_tour2.loc[df_tour2['Trips']<=2, 'IsTrips2'] = '1'
df_tour2['IsPerson1'] = '2'
df_tour2.loc[df_tour2['travelers_total']<=1, 'IsPerson1'] = '1'
df_tour2['IsWorkTour'] = '2'
df_tour2.loc[df_tour2['TOURPURP']==1, 'IsWorkTour'] = '1'
df_tour2.loc[df_tour2['TOURPURP']==1, 'Whyto'] = '11-11'
df_tour2['condition'] =  df_tour2[['IsDist20', 'IsPerson1','IsWorkTour']].agg('-'.join, axis=1)
df_tour2 = df_tour2.merge(df_mode[['condition','probability']], on='condition', how='left')
for i in range(len(df_tour2)): # convert property list from string
    df_tour2['probability'][i] = ast.literal_eval(df_tour2['probability'][i])
df_tour2['bisect'] = 0
for i in range(len(df_tour2)): # imputation
    df_tour2['bisect'][i] = bisect(df_tour2['probability'][i], random.random()) + 1
df_tour2['bisect'] = df_tour2['bisect'].replace(3,5)
df_tour2.loc[df_tour2['HhVehUsed'] == 2, 'Trptrans'] = df_tour2['bisect']
df_tour2.loc[df_tour2['HhVehUsed'] == 2, 'Vehtype'] = df_tour2['Trptrans']
# output
df = df_tour2[['HH_ID','DIST','TravelTime','DwellTime','StartHome', 'EndHome','Trips','travelers_total',\
               'Vehid','Trptrans','Vehtype','HhVehUsed','Whyto','Disttowk']]
df.columns = ['Houseid','Distance','TravelTime','DwellTime','StartHome','EndHome','Trips','Persons','Vehid',\
              'Trptrans','Vehtype','HhVehUsed','Whyto','Disttowk']
# df['Houseid']  = df['Houseid'].apply(lambda x: "'" + str(x) + "'")
list = [['Disttowk',float]]
for item, format in list:
    df[item] = df[item].astype(format)
df.to_csv('ToursByHh_df.csv', index = False)

In [ ]:
# 3 Per_df.csv
df_per = pd.read_csv(_join('HTS', 'per_debug.csv'))[['SAMPN', 'personid', 'PERNO', 'gender', 'AGE_CAT', 'EMPLY', 'license','HHR_RACE']]
df_personABM = pd.read_csv(_join(scenpath, '2016', 'input\persons.csv'))[['perid','age']]
df_per['WORKER'] = 2
df_per.loc[df_per['EMPLY']<=2, 'WORKER'] = 1
df_per['license'] = df_per['license'].fillna(2)
# Assign age group. HTS age group is different to VE age group. Approximate it with the age group proportion in ABM persons.csv
labels = [15, 24, 34,54, 64, 80]
df_per['personBins'] = pd.cut(df_per['AGE_CAT'], bins=driverHTSBin, labels=labels, include_lowest=True)
df_perGro = df_per.groupby(['personBins'])['personid'].count().reset_index(name ='Sum')
age_bin2 = [0,5,14,15,17,18,19,24,29,34,54,64,150] 
age_labels = ['0-5','6-14','15','16-17','18','19','20-24','25-29','30-34','35-54','55-64','65+']
df_personABM['personBins'] = pd.cut(df_personABM['age'], bins=age_bin2, labels=age_labels , include_lowest=True)
df_personABM2 = df_personABM.groupby(['personBins'])['perid'].count().reset_index(name ='Sum')
personABM_list = df_personABM2['Sum'].to_list()
personABMper_list = [] # abm age proportation [0-14, 15, 16, 17, 18, 19,20-24, 25-29, 30-34]
personABMper_list.append(personABM_list[0]/(personABM_list[0]+personABM_list[1]+personABM_list[2]))
personABMper_list.append(personABM_list[1]/(personABM_list[0]+personABM_list[1]+personABM_list[2]))
personABMper_list.append(personABM_list[2]/(personABM_list[0]+personABM_list[1]+personABM_list[2]))
personABMper_list.append(personABM_list[3]/(personABM_list[3]+personABM_list[4]+personABM_list[5]+personABM_list[6]))
personABMper_list.append(personABM_list[4]/(personABM_list[3]+personABM_list[4]+personABM_list[5]+personABM_list[6]))
personABMper_list.append(personABM_list[5]/(personABM_list[3]+personABM_list[4]+personABM_list[5]+personABM_list[6]))
personABMper_list.append(personABM_list[6]/(personABM_list[3]+personABM_list[4]+personABM_list[5]+personABM_list[6]))
personABMper_list.append(personABM_list[7]/(personABM_list[7]+personABM_list[8]))
personABMper_list.append(personABM_list[8]/(personABM_list[7]+personABM_list[8]))
df_per['R_AGE'] = 0
for i in range(len(df_per)): # imputation to more age bins for Hh_df LIF_CYC
    if df_per['personBins'][i] == 15:
        value = bisect([personABMper_list[0],personABMper_list[0]+personABMper_list[1] ,1], random.random())
        if value == 0:
            df_per['R_AGE'][i] = 5
        elif value == 1:
            df_per['R_AGE'][i] = 14                                             
        else: 
            df_per['R_AGE'][i] = 15
    elif df_per['personBins'][i] == 24:
        value = bisect([personABMper_list[3],personABMper_list[3]+personABMper_list[4],\
                        personABMper_list[3]+personABMper_list[4]+personABMper_list[5], 1], random.random())
        if value == 0:
            df_per['R_AGE'][i] = 16
        elif value == 1:
            df_per['R_AGE'][i] = 18
        elif value == 2:
            df_per['R_AGE'][i] = 19
        else: 
            df_per['R_AGE'][i] = 24
    elif df_per['personBins'][i] == 34:
        value = bisect([personABMper_list[7],1], random.random())
        if value == 0:
            df_per['R_AGE'][i] = 29
        else: 
            df_per['R_AGE'][i] = 34      
    else:
        df_per['R_AGE'][i] = df_per['personBins'][i]
# daily number of walk and bike trips
df_trip_rsg['day'] = df_trip_rsg['puid'].str[-1:].astype(int) 
trip_rsg_1day = df_trip_rsg.loc[(df_trip_rsg['day'] == 1) & (df_trip_rsg['TRIPMODE'] == 4)]
trip_rsg_1day_group = trip_rsg_1day.groupby(['HH_ID', 'PER_ID'])['TRIP_ID'].count().reset_index().rename(columns={'TRIP_ID': 'NWALKTRP'}) 
df_per = df_per.merge(trip_rsg_1day_group, left_on=['SAMPN', 'PERNO'], right_on=['HH_ID', 'PER_ID'], how='left')
df_per['NWALKTRP'] = df_per['NWALKTRP'].fillna(0)
df_per['NWALKTRP'] = df_per['NWALKTRP'] * 7 * 1.06
trip_rsg_1dayb = df_trip_rsg.loc[(df_trip_rsg['day'] == 1) & (df_trip_rsg['TRIPMODE'] == 5)]
trip_rsg_1day_groupb = trip_rsg_1dayb.groupby(['HH_ID', 'PER_ID'])['TRIP_ID'].count().reset_index().rename(columns={'TRIP_ID': 'NBIKETRP'}) 
df_per = df_per.merge(trip_rsg_1day_groupb, left_on=['SAMPN', 'PERNO'], right_on=['HH_ID', 'PER_ID'], how='left')
df_per['NBIKETRP'] = df_per['NBIKETRP'].fillna(0)
df_per['NBIKETRP'] = df_per['NBIKETRP'] * 7 * 1.06

In [ ]:
# 3.5 Imputation of weekly bike trips per person
'''
df_bike = pd.read_csv(_join('NHTS2001 analysis', 'bike share.csv'))
df_per['R_AGE_Con'] = '0'
df_per.loc[df_per['R_AGE']<=35, 'R_AGE_Con'] = '1'
df_per['WORKERs'] = df_per['WORKER'].astype(str)
df_per['licenses'] = df_per['license'].astype(int).astype(str)
df_per['genders'] = df_per['gender'].astype(str)
df_per['condition'] =  df_per[['WORKERs', 'licenses','genders','R_AGE_Con']].agg('-'.join, axis=1)
df_per = df_per.merge(df_bike[['condition','probability']], on='condition', how='left')
for i in range(len(df_per)): # convert property list from string
    df_per['probability'][i] = ast.literal_eval(df_per['probability'][i])
df_per['NBIKETRP'] = 0
for i in range(len(df_per)): # imputation
    if df_per['R_AGE'][i] >= 16:
        df_per['NBIKETRP'][i] = bisect(df_per['probability'][i], random.random())
df_per['NBIKETRP'] = df_per['NBIKETRP'].replace({3:4, 4:14})
'''

In [ ]:
# 3.6 Approximate COMMDRVR, USEPUBTR, WRKDRIVE, DTGAS, DISTTOWK - these attributes are not in used in VERSPM modules
df_per['COMMDRVR'] =-1
df_per['USEPUBTR'] = -1
df_per['WRKDRIVE'] = -1
df_per['WRKTRANS'] = -1
df_per['DTGAS'] = -1
df_per['DISTTOWK'] = 0
df_per2 = df_per[['SAMPN','personid','COMMDRVR','NBIKETRP','NWALKTRP','USEPUBTR','WRKDRIVE','WRKTRANS','WORKER',\
             'DTGAS','DISTTOWK','license','R_AGE','gender']]
df_per2.columns = ['HOUSEID','PERSONID','COMMDRVR','NBIKETRP','NWALKTRP','USEPUBTR','WRKDRIVE','WRKTRANS','WORKER',\
              'DTGAS','DISTTOWK','DRIVER','R_AGE','R_SEX']
# df_per2['HOUSEID']  = df_per2['HOUSEID'].apply(lambda x: "'" + str(x) + "'")
list = [['PERSONID',int],['DISTTOWK',float],['DRIVER',int]]
for item, format in list:
    df_per2[item] = df_per2[item].astype(format)
df_per2.to_csv('Per_df.csv', index = False)

In [ ]:
# 4 Veh_df
df_veh = pd.read_csv(_join('HTS', 'SDRTS_Vehicle_Data_20170731(2016HTS).csv'))[['hhid', 'vehnum', 'year', 'category']]
df_veh = df_veh.rename(columns={'hhid':'HOUSEID', 'vehnum':'VEHID', 'year':'VEHYEAR', 'category':'VEHTYPE'})
df_veh['VEHTYPE'] = df_veh['VEHTYPE'].replace({'car': 1, 'light truck': 4, 'Motorcycle': 7, 'OthTrk': 5})
df_veh['BESTMILE'] = -9
df_veh['EIADMPG'] = -9
df_veh['GSCOST'] = -9
df_veh['VEHMILES'] = -9
df_veh = df_veh[['HOUSEID','VEHID','BESTMILE','EIADMPG','GSCOST','VEHTYPE','VEHYEAR','VEHMILES']]
# df_veh['HOUSEID']  = df_veh['HOUSEID'].apply(lambda x: "'" + str(x) + "'")
list = [['BESTMILE',float],['EIADMPG',float],['GSCOST',float]]
for item, format in list:
    df_veh[item] = df_veh[item].astype(format)
df_veh.to_csv('Veh_df.csv', index = False)

In [ ]:
# 5 Hh_df - This module must be run together and AFTER Per_df and Veh_df!!!
df_hhhts = pd.read_csv(_join('HTS', 'SDRTS_Household_Data_20170731 TAZ.csv'))
df_hhhts_mgra = pd.read_csv(_join('HTS', 'SDRTS_Household_Data_20170731 MGRA.csv'))
df_area = pd.read_csv(_join(target_path, 'inputs', 'bzone_unprotected_area.csv'))
df_mgra = pd.read_csv(_join(scenpath, '2016', 'input\mgra13_based_input' + '2016'+ '.csv'))
df_cbg = pd.read_csv(_join('HTS', 'mgra13_CT10BG.csv'))
df_areatype = pd.read_csv(_join('HTS', 'areatype_mgra.csv'), engine='python')
df_income = pd.read_csv(_join('NHTS2001 analysis', 'hhincome share.csv'))

df_countdriver = df_per.groupby(['SAMPN','license'])['personid'].count().reset_index()
df_countdriver = df_countdriver.loc[df_countdriver['license'] == 1.0]
df_countdriver = df_countdriver.rename(columns={'SAMPN': 'hhid', 'personid': 'DRVRCNT'})
df_hhhts = df_hhhts.merge(df_countdriver[['hhid', 'DRVRCNT']], on='hhid', how='left')
df_housecount = df_hhhts.groupby('home_address_TAZ')['hh_final_weight_456x'].sum().reset_index()
df_area = df_area.loc[df_area['Year'] == 2016]
df_area['area'] = (df_area['RuralArea']+df_area['TownArea']+df_area['UrbanArea']) * 0.0015625
df_mgraTAZ = df_mgra.groupby('taz')['hs'].sum().reset_index()
df_areaHh = df_area.merge(df_mgraTAZ, left_on='Geo', right_on='taz', how='left')
df_per = df_per.merge(df_hhhts[['hhid','home_address_MGRA','hh_final_weight_456x']], left_on='SAMPN', right_on='hhid', how='left')

df_mgra = df_mgra.merge(df_cbg, left_on='mgra', right_on='MGRA', how='left' )
df_mgraden = df_mgra.groupby('CT10BG')['hs', 'pop', 'land_acres'].sum().reset_index()
df_mgraden['popden'] = df_mgraden['pop']/(df_mgraden['land_acres'] * 0.0015625)
df_mgraden['popden'] = df_mgraden['popden'].fillna(0)
areaPer_bin = [0, 99,499,999,1999,3999,9999,24999,1000000000000] 
areaPer_labels = [50,300,750,1500,3000,7000,17000,30000]
df_mgraden['HBPPOPDN']  = pd.cut(df_mgraden['popden'], bins=areaPer_bin, labels=areaPer_labels, include_lowest=True)
df_mgraden['hsden'] = df_mgraden['hs']/(df_mgraden['land_acres'] * 0.0015625)
df_mgraden['hsden'] = df_mgraden['hsden'].fillna(0)
area_bin = [0, 49,249,999,2999,4999,100000000] 
area_labels = [25,150,700,2000,4000,6000]
df_mgraden['HBHRESDN'] = pd.cut(df_mgraden['hsden'], bins=area_bin, labels=area_labels, include_lowest=True)
df_mgra = df_mgra.merge(df_mgraden[['CT10BG','HBPPOPDN','HBHRESDN']], on='CT10BG', how='left')
df_hhhts = df_hhhts.merge(df_mgra[['mgra', 'HBHRESDN', 'HBPPOPDN']], left_on='home_address_MGRA', right_on='mgra', how='left')
df_hhhts = df_hhhts.merge(df_areatype[['MGRA', 'HBHUR']], left_on='home_address_MGRA', right_on='MGRA', how='left')

In [ ]:
# 6 Imputation of household income
hhsize_bin = [0,1,3,100] 
hhsize_labels = ['A','B','C']
df_hhhts['hhsizeGrp'] = pd.cut(df_hhhts['hhsize'], bins=hhsize_bin, labels=hhsize_labels, include_lowest=True)
wksize_bin = [0,0.1,1,100] 
wksize_labels = ['0','1','2+']
df_hhhts['wksizeGrp'] = pd.cut(df_hhhts['numworkers'], bins=wksize_bin, labels=wksize_labels, include_lowest=True)
df_hhhts['hhsizeGrp'] = df_hhhts['hhsizeGrp'].astype(str)+'-'+df_hhhts['wksizeGrp'].astype(str)
df_hhhts = df_hhhts.merge(df_income, left_on = 'hhsizeGrp', right_on='condition', how='left')
df_hhhts['hhincome_imputed'] = df_hhhts['hhincome_imputed'].fillna(df_hhhts['hhincome_broad'])
for i in range(len(df_hhhts)): # convert property list from string
    df_hhhts['group1'][i] = ast.literal_eval(df_hhhts['group1'][i])
    df_hhhts['group2'][i] = ast.literal_eval(df_hhhts['group2'][i])
    df_hhhts['group3'][i] = ast.literal_eval(df_hhhts['group3'][i])
df_hhhts['hhincome_imputed2'] = 0
for i in range(len(df_hhhts)): # imputation
    if df_hhhts['hhincome_imputed'][i] == 1:
        df_hhhts['hhincome_imputed2'][i] = bisect(df_hhhts['group1'][i], random.random()) + 1
    elif df_hhhts['hhincome_imputed'][i] == 2:
        df_hhhts['hhincome_imputed2'][i] = bisect(df_hhhts['group2'][i], random.random()) + 7  
    elif df_hhhts['hhincome_imputed'][i] == 3:
        df_hhhts['hhincome_imputed2'][i] = bisect(df_hhhts['group3'][i], random.random()) + 13      
    elif df_hhhts['hhincome_imputed'][i] in [4,5]:
        df_hhhts['hhincome_imputed2'][i] = 18
    elif df_hhhts['hhincome_imputed'][i] == 99:
        df_hhhts['hhincome_imputed2'][i] = -9
    else:
        print ('exception', df_hhhts['hhid'][i], df_hhhts['hhincome_imputed'][i])

In [ ]:
# 7
df_per2 = df_per.loc[df_per['PERNO']==1] # approximation, use the first person in HTS households as the respondent
df_hhhts = df_hhhts.merge(df_per2[['SAMPN', 'R_AGE', 'license', 'HHR_RACE','gender']], left_on='hhid', right_on='SAMPN', how='left')
df_veh2 = df_veh.groupby('HOUSEID')['VEHID'].count().reset_index()
#df_hhhts['hhid']  = df_hhhts['hhid'].apply(lambda x: "'" + str(x) + "'")
df_hhhts =df_hhhts.merge(df_veh2, left_on='hhid', right_on='HOUSEID', how='left')
df_hhhts['res_type'] = df_hhhts['res_type'].replace({3: 4, 97: 91})
# impute HTS 2-Townhouse (attached house) to NHTS 2-Duplex 6 and 3-Rowhouse or townhouse 6 [0.537, 0.463]
row_list =  [0.537, 1]
for i in range(len(df_hhhts)): # imputation
    if df_hhhts['res_type'][i] == 2:
        df_hhhts['res_type'][i] = bisect(row_list, random.random()) + 2

In [ ]:
# 8 assign life cycle, adult: 18 and plus
df_per['LIF_CYC'] = 0
for hhid in df_per['SAMPN'].unique():
    adult = adult_retire = child5 = child15 = child17 = 0
    df_select = df_per.loc[df_per['SAMPN'] == hhid]
    adult = len(df_select.loc[(df_select['R_AGE']>=18) & (df_select['EMPLY']!=4)]) 
    adult_retire = len(df_select.loc[(df_select['R_AGE']>=18) & (df_select['EMPLY']==4)]) 
    child5 = len(df_select.loc[df_select['R_AGE']<=5])
    child15 = len(df_select.loc[(df_select['R_AGE']>=5) & (df_select['R_AGE']<=15)])
    child17 = len(df_select.loc[(df_select['R_AGE']>=16) & (df_select['R_AGE']<=17)])
    if adult_retire == 1 and adult == 0 and child5+child15+child17==0:
        df_per.loc[df_per['SAMPN'] == hhid, ['LIF_CYC']] = 9
    elif adult_retire == 2 and adult == 0 and child5+child15+child17==0:
        df_per.loc[df_per['SAMPN'] == hhid, ['LIF_CYC']] = 10
    else:
        if adult + adult_retire == 1:
            if child5 > 0:
                df_per.loc[df_per['SAMPN'] == hhid, ['LIF_CYC']] = 3
            elif child15 > 0:
                df_per.loc[df_per['SAMPN'] == hhid, ['LIF_CYC']] = 5
            elif child17 > 0:
                df_per.loc[df_per['SAMPN'] == hhid, ['LIF_CYC']] = 7
            else:
                df_per.loc[df_per['SAMPN'] == hhid, ['LIF_CYC']] = 1
        else:
            if child5 > 0:
                df_per.loc[df_per['SAMPN'] == hhid, ['LIF_CYC']] = 4
            elif child15 > 0:
                df_per.loc[df_per['SAMPN'] == hhid, ['LIF_CYC']] = 6
            elif child17 > 0:
                df_per.loc[df_per['SAMPN'] == hhid, ['LIF_CYC']] = 8
            else:
                df_per.loc[df_per['SAMPN'] == hhid, ['LIF_CYC']] = 2
df_per3 = df_per[df_per['PERNO'] == 1]
df_hhhts = df_hhhts.merge(df_per3[['SAMPN','LIF_CYC']], on ='SAMPN', how ='left')

In [ ]:
# 9
df_hhhts['MSAPOP'] = [df_per['hh_final_weight_456x'].sum()] * len(df_hhhts)
df_per4 = df_per2.loc[df_per['R_AGE']>=16]
df_per5 = df_per4.groupby('SAMPN')['PERNO'].count().reset_index().rename(columns={'PERNO': 'PER16+'}) 
df_hhhts = df_hhhts.merge(df_per5[['SAMPN','PER16+']], on='SAMPN', how='left')
df_hhhts['RATIO16V'] = df_hhhts['PER16+'] / df_hhhts['VEHID']
df_hhhts['URBAN'] = 2
df_hhhts.loc[df_hhhts['HBHUR']=='R', 'URBAN'] = 4
df_hhhts['URBRUR'] = 1
df_hhhts.loc[df_hhhts['HBHUR']=='R', 'URBRUR'] = 2
df_per6 = df_per.loc[df_per['WORKER']==1]
df_per7 = df_per6.groupby('SAMPN')['PERNO'].count().reset_index().rename(columns={'PERNO': 'WRKCOUNT'}) 
df_hhhts = df_hhhts.merge(df_per7[['SAMPN','WRKCOUNT']], on='SAMPN', how='left')

In [ ]:
# 10 assign age related AGE_, DRV_, WKR_
df_perage = df_per.pivot_table(values='R_AGE', index=['SAMPN'], columns=['PERNO'], aggfunc=np.sum)
df_perage = df_perage.fillna(-1)
df_perage[13] = df_perage[14] = -1
df_perage = df_perage.astype(int)
df_perage.columns = ['AGE_P1','AGE_P2','AGE_P3','AGE_P4','AGE_P5','AGE_P6','AGE_P7','AGE_P8','AGE_P9','AGE_P10','AGE_P11','AGE_P12','AGE_P13','AGE_P14']

df_perdr = df_per.pivot_table(values='license', index=['SAMPN'], columns=['PERNO'], aggfunc=np.sum)
df_perdr = df_perdr.fillna(-1)
df_perdr[13] = df_perdr[14] = -1
df_perdr = df_perdr.astype(int)
df_perdr.columns = ['DRV_P1','DRV_P2','DRV_P3','DRV_P4','DRV_P5','DRV_P6','DRV_P7','DRV_P8','DRV_P9','DRV_P10','DRV_P11','DRV_P12','DRV_P13','DRV_P14']

df_perwr = df_per.pivot_table(values='WORKER', index=['SAMPN'], columns=['PERNO'], aggfunc=np.sum)
df_perwr = df_perwr.fillna(-1)
df_perwr[13] = df_perwr[14] = -1
df_perwr = df_perwr.astype(int)
df_perwr.columns = ['WKR_P1','WKR_P2','WKR_P3','WKR_P4','WKR_P5','WKR_P6','WKR_P7','WKR_P8','WKR_P9','WKR_P10','WKR_P11','WKR_P12','WKR_P13','WKR_P14']
df_perall = df_perage.merge(df_perdr, on='SAMPN', how='inner')
df_perall = df_perall.merge(df_perwr, on='SAMPN', how='inner')
df_hhhts = df_hhhts.merge(df_perall, on='SAMPN', how='left')

In [ ]:
# 11 assign MSACAT
df_SDMSA = pd.read_csv(_join(r'T:\ABM\ABM_FY22\RSM\VisionEval\Model\Update_Automations\SX', 'mgra13_msacat.csv'))
df_hhhts = df_hhhts.merge(df_SDMSA[['mgra', 'Msacat', 'HHC_MSA']], left_on='home_address_MGRA', right_on='mgra', how='left')

In [ ]:
# 12 approximations/placeholders
df_hhhts['FLGFINCM'] = 1 # assume incomes of all HH members are included
df_hhhts['CENSUS_D'] = 9
df_hhhts['CENSUS_R'] = 4
# df_hhhts['HHC_MSA'] = '7320'
df_hhhts['HHINCTTL'] = df_hhhts['hhincome_imputed2']
df_hhhts['HHNUMBIK'] = 0
df_hhhts['HTEEMPDN'] = 0
df_hhhts['HTHRESDN'] = df_hhhts['HBHRESDN']
df_hhhts['HTHUR'] = df_hhhts['HBHUR']
df_hhhts['HTPPOPDN'] = df_hhhts['HBPPOPDN']
#df_hhhts['MSACAT'] = 2
df_hhhts['MSASIZE'] = 5
df_hhhts['RAIL'] = 2
df_hhhts['CNTTDHH'] = 0

In [ ]:
# 13
df_hhhts2 = df_hhhts[['hhid','AGE_P1','AGE_P2','AGE_P3','AGE_P4','AGE_P5','AGE_P6','AGE_P7','AGE_P8','AGE_P9','AGE_P10',\
                     'AGE_P11','AGE_P12','AGE_P13','AGE_P14','CENSUS_D','CENSUS_R','DRVRCNT','DRV_P1','DRV_P2','DRV_P3',\
                     'DRV_P4','DRV_P5','DRV_P6','DRV_P7','DRV_P8','DRV_P9','DRV_P10','DRV_P11','DRV_P12','DRV_P13',\
                     'DRV_P14','hh_final_weight_456x','hh_final_weight_456x','FLGFINCM','HBHRESDN','HBHUR','HBPPOPDN',\
                     'HHC_MSA','hhincome_imputed2','HHINCTTL','HHNUMBIK','R_AGE','license','HHR_RACE','gender','hhsize',\
                     'VEHID','res_type','HTEEMPDN','HTHRESDN','HTHUR','HTPPOPDN','LIF_CYC','MSAPOP','Msacat','MSASIZE',\
                     'RAIL','RATIO16V','URBAN','URBRUR','WRKCOUNT','WKR_P1','WKR_P2','WKR_P3','WKR_P4','WKR_P5','WKR_P6',\
                     'WKR_P7','WKR_P8','WKR_P9','WKR_P10','WKR_P11','WKR_P12','WKR_P13','WKR_P14','CNTTDHH']]
df_hhhts2.columns = ['HOUSEID','AGE_P1','AGE_P2','AGE_P3','AGE_P4','AGE_P5','AGE_P6','AGE_P7','AGE_P8','AGE_P9','AGE_P10',\
                     'AGE_P11','AGE_P12','AGE_P13','AGE_P14','CENSUS_D','CENSUS_R','DRVRCNT','DRV_P1','DRV_P2','DRV_P3',\
                     'DRV_P4','DRV_P5','DRV_P6','DRV_P7','DRV_P8','DRV_P9','DRV_P10','DRV_P11','DRV_P12','DRV_P13',\
                     'DRV_P14','EXPFLHHN','EXPFLLHH','FLGFINCM','HBHRESDN','HBHUR','HBPPOPDN','HHC_MSA','HHFAMINC',\
                     'HHINCTTL','HHNUMBIK','HHR_AGE','HHR_DRVR','HHR_RACE','HHR_SEX','HHSIZE','HHVEHCNT','HOMETYPE',\
                     'HTEEMPDN','HTHRESDN','HTHUR','HTPPOPDN','LIF_CYC','MSAPOP','MSACAT','MSASIZE','RAIL','RATIO16V',\
                     'URBAN','URBRUR','WRKCOUNT','WKR_P1','WKR_P2','WKR_P3','WKR_P4','WKR_P5','WKR_P6','WKR_P7','WKR_P8',\
                     'WKR_P9','WKR_P10','WKR_P11','WKR_P12','WKR_P13','WKR_P14','CNTTDHH']
# df_hhhts2['HHC_MSA']  = df_hhhts2['HHC_MSA'].apply(lambda x: "'" + str(x) + "'")
df_hhhts2['DRVRCNT'] = df_hhhts2['DRVRCNT'].fillna(0)
list = [['DRVRCNT',int],['HHR_DRVR',int],['HHVEHCNT',int],['MSAPOP',int],['WRKCOUNT',int]]
for item, format in list:
    df_hhhts2[item] = df_hhhts2[item].fillna(0)
    df_hhhts2[item] = df_hhhts2[item].astype(format)
df_hhhts2.to_csv('Hh_df.csv', index = False)

In [ ]:
# assign unique tourID to RSG Tour file
df_tour = pd.read_csv(_join('HTS', 'tours_debug.csv'))
df = pd.DataFrame()
for hhid in df_tour['HH_ID'].unique():
    for perid in df_tour.loc[(df_tour['HH_ID'] == hhid)]['PER_ID'].unique():
        df_select = df_tour.loc[(df_tour['HH_ID'] == hhid) & (df_tour['PER_ID'] == perid)]
        l = [i for i in range(1,len(df_select)+1,1)]
        df_select = df_select.set_index([pd.Index(l)])
        df_select = df_select.reset_index().rename({'index':'TOUR_ID2'}, axis = 'columns')
        df =df.append(df_select)
df.to_csv('tours_debug_newid.csv', index = False)

In [ ]:
# assign unique tourID to RSG Trip File
df_trip = pd.read_csv(_join('HTS', 'trips_debug.csv'))
df_ntrip = pd.DataFrame()
#j=0
for hhid in df_trip['HH_ID'].unique():
    #if hhid == 161000914:
        for perid in df_trip.loc[(df_trip['HH_ID'] == hhid)]['PER_ID'].unique():        
            df_selectTrip = df_trip.loc[(df_trip['HH_ID'] == hhid) & (df_trip['PER_ID'] == perid)]
            #print ('df_selectTrip aaa')
            #print (df_selectTrip[['HH_ID','PER_ID','TOUR_ID','TRIP_ID','ORIG_PLACENO']], len(df_selectTrip))
            l = [i for i in range(0,len(df_selectTrip),1)]
            df_selectTrip = df_selectTrip.set_index([pd.Index(l)])
            tourid = 0
            for i in range(len(df_selectTrip)):
                if df_selectTrip.iloc[i]['TRIP_ID'] == 1:
                    tourid = tourid + 1
                    #print ('if i, tourid', i, tourid)
                    #print ('df_selectTrip.iloc[i]', df_selectTrip.iloc[i])
                    #print ('df_selectTrip ccc')
                    #print (df_selectTrip[['HH_ID','PER_ID','TOUR_ID','TRIP_ID','ORIG_PLACENO']])       
                #print ('else i, tourid', i, tourid)
                df_selectTrip.at[i,'TOUR_ID'] = tourid
                #print ('df_selectTrip ddd')
                #print (df_selectTrip[['HH_ID','PER_ID','TOUR_ID','TRIP_ID','ORIG_PLACENO']])                       
                #print ('i', i)
            #print ('df_selectTrip bbb, i', i)
            #print (df_selectTrip[['HH_ID','PER_ID','TOUR_ID','TRIP_ID','ORIG_PLACENO']])                
            df_ntrip = df_ntrip.append(df_selectTrip)
    #else:
    #    continue
    #j += 1
df_ntrip.to_csv('trips_debug_newid', index = False)